In [118]:
import torch
import pandas as pd
from transformers import BlenderbotTokenizer, BlenderbotForCausalLM, BlenderbotConfig
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import tqdm



In [71]:
tokenizer = BlenderbotTokenizer.from_pretrained("facebook/blenderbot-400M-distill")

model_config = BlenderbotConfig.from_pretrained("facebook/blenderbot-400M-distill")

model = BlenderbotForCausalLM.from_pretrained("facebook/blenderbot-400M-distill", config=model_config)

Some weights of the model checkpoint at facebook/blenderbot-400M-distill were not used when initializing BlenderbotForCausalLM: ['model.encoder.layers.0.self_attn_layer_norm.weight', 'model.encoder.layers.0.self_attn_layer_norm.bias', 'model.encoder.layers.1.self_attn.q_proj.weight', 'model.encoder.layers.0.self_attn.out_proj.bias', 'model.encoder.layers.0.self_attn.out_proj.weight', 'model.encoder.layers.0.fc1.bias', 'model.encoder.layer_norm.weight', 'model.encoder.layers.1.final_layer_norm.bias', 'model.encoder.layers.0.fc2.bias', 'model.encoder.layers.0.final_layer_norm.weight', 'model.encoder.layers.0.fc2.weight', 'model.encoder.layers.1.self_attn.v_proj.weight', 'model.encoder.layers.1.fc1.bias', 'model.shared.weight', 'model.encoder.layers.0.self_attn.v_proj.weight', 'model.encoder.layers.1.self_attn_layer_norm.bias', 'model.encoder.embed_positions.weight', 'model.encoder.layers.0.self_attn.q_proj.weight', 'model.encoder.layers.0.self_attn.v_proj.bias', 'model.encoder.layers.0.s

In [93]:
df = pd.read_csv("/Users/damodargupta/Desktop/sentence-emotion analysis model- psyaa/fine-tuning-blenderbot/train.csv",  error_bad_lines=False)
df

/var/folders/pb/3pqt97t1701c0435w57nfgrh0000gn/T/ipykernel_30597/3380662777.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("/Users/damodargupta/Desktop/sentence-emotion analysis model- psyaa/fine-tuning-blenderbot/train.csv",  error_bad_lines=False)
Skipping line 2355: expected 8 fields, saw 10
Skipping line 36628: expected 8 fields, saw 12
Skipping line 49433: expected 8 fields, saw 10
Skipping line 56957: expected 8 fields, saw 10
Skipping line 65019: expected 8 fields, saw 10



,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval
hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,NaN
hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,NaN
hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,NaN
hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,NaN
hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,NaN
...,...,...,...,...,...,...,...
hit:12424_conv:24848,5,sentimental,I found some pictures of my grandma in the att...,389,Yeah reminds me of the good old days. I miss ...,5|5|5_5|5|5,NaN
hit:12424_conv:24849,1,surprised,I woke up this morning to my wife telling me s...,294,I woke up this morning to my wife telling me s...,5|5|5_5|5|5,NaN
hit:12424_conv:24849,2,surprised,I woke up this morning to my wife telling me s...,389,Oh hey that's awesome! That is awesome right?,5|5|5_5|5|5,NaN
hit:12424_conv:24849,3,surprised,I woke up this morning to my wife telling me s...,294,It is soooo awesome. We have been wanting a b...,5|5|5_5|5|5,NaN


In [100]:
df=df.drop('selfeval',axis=1)

In [101]:
df

,utterance_idx,context,speaker_idx
hit:0_conv:1,sentimental,I remember going to the fireworks with my best...,I remember going to see the fireworks with my ...
hit:0_conv:1,sentimental,I remember going to the fireworks with my best...,Was this a friend you were in love with_comma_...
hit:0_conv:1,sentimental,I remember going to the fireworks with my best...,This was a best friend. I miss her.
hit:0_conv:1,sentimental,I remember going to the fireworks with my best...,Where has she gone?
hit:0_conv:1,sentimental,I remember going to the fireworks with my best...,We no longer talk.
...,...,...,...
hit:12424_conv:24848,sentimental,I found some pictures of my grandma in the att...,Yeah reminds me of the good old days. I miss ...
hit:12424_conv:24849,surprised,I woke up this morning to my wife telling me s...,I woke up this morning to my wife telling me s...
hit:12424_conv:24849,surprised,I woke up this morning to my wife telling me s...,Oh hey that's awesome! That is awesome right?
hit:12424_conv:24849,surprised,I woke up this morning to my wife telling me s...,It is soooo awesome. We have been wanting a b...


In [102]:
training_text = df['utterance_idx'] + ' ' + df['context'] + ' ' + df['speaker_idx']
training_text


hit:0_conv:1            sentimental I remember going to the fireworks ...
hit:0_conv:1            sentimental I remember going to the fireworks ...
hit:0_conv:1            sentimental I remember going to the fireworks ...
hit:0_conv:1            sentimental I remember going to the fireworks ...
hit:0_conv:1            sentimental I remember going to the fireworks ...
                                              ...                        
hit:12424_conv:24848    sentimental I found some pictures of my grandm...
hit:12424_conv:24849    surprised I woke up this morning to my wife te...
hit:12424_conv:24849    surprised I woke up this morning to my wife te...
hit:12424_conv:24849    surprised I woke up this morning to my wife te...
hit:12424_conv:24849    surprised I woke up this morning to my wife te...
Length: 76668, dtype: object

In [103]:
df = pd.read_csv('/Users/damodargupta/Desktop/sentence-emotion analysis model- psyaa/training text.csv')
df = df.drop('Unnamed: 0' , axis=1)
df.to_csv('/Users/damodargupta/Desktop/sentence-emotion analysis model- psyaa/training text.csv' , index=False)



In [104]:
df

,0
0,I remember going to the fireworks with my best...
1,I remember going to the fireworks with my best...
2,I remember going to the fireworks with my best...
3,I remember going to the fireworks with my best...
4,I remember going to the fireworks with my best...
...,...
76663,I found some pictures of my grandma in the att...
76664,I woke up this morning to my wife telling me s...
76665,I woke up this morning to my wife telling me s...
76666,I woke up this morning to my wife telling me s...


In [84]:
df['prompt']=df['prompt'].astype(str)
df['prompt'].dtype

dtype('O')

In [113]:
input_ids = tokenizer.batch_encode_plus(
    training_text.tolist(),
    padding=True,
    truncation=True,
    max_length=512,  
    return_tensors="pt"
)["input_ids"]

In [119]:
dataset = TextDataset(input_ids, file_path='/Users/damodargupta/Desktop/sentence-emotion analysis model- psyaa/training text.csv' , block_size= 64)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

AttributeError: 'Tensor' object has no attribute 'num_special_tokens_to_add'

In [ ]:
training_args = TrainingArguments(
    output_dir="/Users/damodargupta/Desktop/sentence-emotion analysis model- psyaa/fine-tuning-blenderbot/fine_tuned_model",
    overwrite_output_dir=True,
    num_train_epochs=10,  
    per_device_train_batch_size=4,  
    save_steps=500, 
    save_total_limit=2, 
    prediction_loss_only=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
with tqdm(total=training_args.num_train_epochs, desc="Training") as pbar:
    def update_pbar_callback(eng):
        pbar.update()
    trainer.add_event_handler(Trainer.EPOCH_COMPLETED, update_pbar_callback)
    trainer.train()

In [ ]:
trainer.save_model("/Users/damodargupta/Desktop/sentence-emotion analysis model- psyaa/fine-tuning-blenderbot/fine_tuned_model")